In [85]:
## Let's try to figure out who the shortest, heaviest baseball players are.

## Here's our overall plan of attack

# Setting Things Up ✅
## Import CSVs ✅
### Separate CSVs --> DataFrames for People, Pitching Data, Batting Data ✅
## Squish everything into one mondo DF ✅
## Add Physical Data ✅
### Height ✅
### Weight ✅
## Calculate BMI ✅
### Convert Imperial to Metric ✅
### BMI-ify ✅
### Throw BMI back into df ✅
## Calculate mean fWAR/bWAR (mWAR)
## Assemble per-position lists sorted by BMI, then mWAR

# Knocking Things Down
## Find worst team that made playoffs in 2021
### Describe team fWAR/bWAR
### Describe individual fWAR/bWAR
## Pull from BMI lists per position until high BMI roster is full
### mWAR shall be higher on a team basis.
### mWAR shall be higher per position.
# If there's enough time:
## Repeat for:
### Tallness
### Shortness
### Heavy
### Light

# If there's enough time:
## Repeat for:
### Tallness
### Shortness
### Heavy
### Light

## Setting Things Up

### Import the necessaries

In [86]:
import pandas as pd
import pybaseball
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


In [87]:
chadwick = chadwick_register(save=True)
chadwick = pd.DataFrame(chadwick)



In [88]:
download_lahman()

## Read infinity baseball data to DataFrames

In [89]:
# a table of all player biographical info and ids
people = pd.DataFrame(people())

# park id, name, alias, city, state, and country
parks = pd.DataFrame(parks())

# all star roster data: player, year, team, league, position
allstar = pd.DataFrame(all_star_full())

# each player's games played per position for each season
appearances = pd.DataFrame(appearances())

# batting stats by year, regular season
batting = pd.DataFrame(batting())

# batting stats by year, post season
batting_post = pd.DataFrame(batting_post())

# fielding stats by year 
fielding = pd.DataFrame(fielding())

# games played in left, center, right field 
fielding_of = pd.DataFrame(fielding_of())

# LF/CF/RF splits
fielding_of_split = pd.DataFrame(fielding_of_split())

# postseason fielding 
fielding_post = pd.DataFrame(fielding_post())

# home game attendance by park by year 
home_games = pd.DataFrame(home_games())

# historical player pitching stats
pitching = pd.DataFrame(pitching())

# postseason pitching stats
pitching_post = pd.DataFrame(pitching_post())

# playoff series winners and losers 
series_post = pd.DataFrame(series_post())

# data on teams by year: record, division, stadium, attendance, etc
teams = pd.DataFrame(teams())

# current and historical franchises, whether they're still active, and their ids
teams_franchises = pd.DataFrame(teams_franchises())

# split season data for teams
teams_half = pd.DataFrame(teams_half()) 

# fangraphs batting since 2008
fangraphs_batting = pd.DataFrame(pybaseball.batting_stats_range(
    start_dt="2008-01-01", end_dt="2021-12-31"))

# fangraphs pitching since 2008
fangraphs_pitching = pd.DataFrame(pybaseball.pitching_stats_range(
    start_dt="2008-01-01", end_dt="2021-12-31"))

# fangraphs team pitching since 2008
fangraphs_team_pitching = pd.DataFrame(
    pybaseball.team_pitching(start_season="2008", end_season="2021"))

# fangraphs team batting since 2008
fangraphs_team_batting = pd.DataFrame(
    pybaseball.team_batting(start_season="2008", end_season="2021"))

# fangraphs team fielding since 2008
fangraphs_team_fielding = pd.DataFrame(
    pybaseball.team_fielding(start_season="2008", end_season="2021"))

# bref pitching WAR
bwar_pitch = pd.DataFrame(bwar_pitch(return_all=True))

# bref batting WAR
bwar_bat = pd.DataFrame(bwar_bat(return_all=True))


/Users/bean/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pybaseball/league_batting_stats.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


In [90]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,2,2,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,3,0,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [91]:
len(bwar_pitch['player_ID'].unique()) + len(bwar_bat['player_ID'].unique())


34002

In [92]:
bwar_pitch

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,11,0,32,0.0,32.0,8,5.967,-0.440,NaN,-0.423,101,98.525,5.862,44.0,0.0093,-45.5,-2.138,-2.177,-1.199,0.203,1.0718,-0.15,300000.0,4.67092,4.86883,1.902,0.4803,-0.2167,-0.0298,4.75633,1.895,0.4914,0.0942,66.675000,5.334
1,David Aardsma,24.0,430911.0,aardsda01,2006,CHC,1,NL,45,0,159,0.0,159.0,25,28.172,-2.076,NaN,-0.367,103,104.018,27.526,147.0,0.0337,-10.9,2.526,2.751,7.388,0.211,0.7542,0.61,NaN,4.86675,4.80561,1.909,0.5060,0.2700,-0.1301,4.97303,1.919,0.4896,0.4682,114.295833,27.431
2,David Aardsma,25.0,430911.0,aardsda01,2007,CHW,1,AL,25,0,97,0.0,97.0,24,17.999,-1.343,NaN,-1.327,104,103.205,18.559,97.0,0.0206,-64.4,-5.441,-5.646,-1.814,0.218,1.4280,-0.40,387500.0,4.85895,5.08479,1.924,0.4782,-0.5450,-0.1990,5.00394,1.920,0.4859,0.3459,74.086957,17.040
3,David Aardsma,26.0,430911.0,aardsda01,2008,BOS,1,AL,47,0,146,0.0,146.0,32,26.131,-1.942,NaN,0.663,106,106.614,25.082,139.0,0.0317,20.9,-6.918,-7.757,-1.633,0.211,0.7972,-0.31,403250.0,4.70400,4.86905,1.904,0.4836,-0.7708,-0.0632,4.81624,1.901,0.4888,0.5205,83.796667,25.139
4,David Aardsma,27.0,430911.0,aardsda01,2009,SEA,1,AL,73,0,214,0.0,214.0,23,39.221,-2.895,NaN,3.136,95,93.595,31.064,181.0,0.0399,78.6,8.064,7.689,15.965,0.215,1.8922,1.69,419000.0,4.79788,4.69256,1.899,0.5105,0.7665,0.1577,4.90529,1.911,0.4894,0.7666,168.910000,33.782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53650,Stephen Strasburg,33.0,544931.0,strasst01,2022,WSN,1,NL,1,1,14,14.0,0.0,7,2.219,0.103,0.0,-0.245,95,98.000,2.516,15.0,0.0070,-35.0,-4.484,-4.561,-4.042,0.200,1.0000,-0.27,35000000.0,4.54287,9.10387,2.106,0.1879,-0.3121,-0.0052,4.97855,1.901,0.4566,0.0440,31.914286,2.234
53651,Jackson Tetreault,26.0,676194.0,tetreja01,2022,WSN,1,NL,3,3,51,51.0,0.0,11,8.718,0.403,0.0,-1.012,95,93.067,9.430,62.0,0.0289,-35.0,-1.570,-1.850,0.041,0.200,1.0000,-0.04,NaN,4.54287,5.15954,1.911,0.4395,-0.1815,-0.0188,5.07191,1.906,0.4477,0.1590,96.125000,7.690
53652,Mason Thompson,24.0,666168.0,thompma02,2022,WSN,1,NL,3,0,6,0.0,6.0,0,1.185,-0.079,0.0,-0.067,95,90.000,1.056,4.0,0.0019,-35.0,1.056,1.023,1.245,0.200,0.1067,0.07,NaN,4.54287,4.20187,1.855,0.5361,0.1083,-0.0559,4.60511,1.879,0.4936,0.0194,NaN,1.055
53653,Austin Voth,30.0,608723.0,vothau01,2022,WSN,1,NL,19,0,56,0.0,56.0,22,9.456,-0.627,0.0,-1.141,95,96.340,9.605,70.0,0.0326,-35.0,-12.395,-12.703,-10.626,0.200,0.4653,-0.80,875000.0,4.54287,5.21145,1.914,0.4347,-1.2407,0.2613,4.63459,1.881,0.4906,0.1809,40.128571,8.427


In [93]:
chadwick.shape

(24258, 8)

In [94]:
df = pd.merge(chadwick, bwar_pitch, how='left', left_on='key_bbref', right_on='player_ID')

In [95]:
df = pd.merge(df, bwar_bat, how='left', left_on='key_bbref', right_on='player_ID')


In [96]:
df.columns

Index(['name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_bbref',
       'key_fangraphs', 'mlb_played_first', 'mlb_played_last', 'name_common_x',
       'age_x', 'mlb_ID_x', 'player_ID_x', 'year_ID_x', 'team_ID_x',
       'stint_ID_x', 'lg_ID_x', 'G_x', 'GS', 'IPouts', 'IPouts_start',
       'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj',
       'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc',
       'RS_def_total', 'runs_above_avg_x', 'runs_above_avg_adj',
       'runs_above_rep_x', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR_x',
       'salary_x', 'teamRpG_x', 'oppRpG_x', 'pyth_exponent_x',
       'waa_win_perc_x', 'WAA_x', 'WAA_adj', 'oppRpG_rep_x',
       'pyth_exponent_rep_x', 'waa_win_perc_rep_x', 'WAR_rep_x', 'ERA_plus',
       'ER_lg', 'name_common_y', 'age_y', 'mlb_ID_y', 'player_ID_y',
       'year_ID_y', 'team_ID_y', 'stint_ID_y', 'lg_ID_y', 'PA', 'G_y', 'Inn',
       'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_inf

In [97]:
df = df[df['year_ID_y'] >= 1921]
# Throw out dead ball era

 7.48       1
-2.80       1
 7.66       1
-3.01       2
 8.90       2
         ... 
 0.02    2909
 0.04    3074
 0.01    3105
-0.02    3118
 0.03    3339
Name: WAR_x, Length: 1168, dtype: int64

### Set constants we'll need

In [113]:
KG_TO_LB = 0.453592
M_TO_IN = 0.0254

### Squish everything into one mondo DF

In [121]:
df_pitchers = pd.concat([df_people, df_pitching], join='inner', axis=1)

In [129]:
frames = [df_batters, df_pitchers]

In [130]:
for player in frames:
    player['KG'] = player['weight'] * KG_TO_LB
    player['meters'] = player['height'] * M_TO_IN
    player['BMI'] = player['KG'] / player['meters'] ** 2

In [131]:
df_batters.dtypes

playerID         object
birthYear       float64
birthMonth      float64
birthDay        float64
birthCountry     object
birthState       object
birthCity        object
deathYear       float64
deathMonth      float64
deathDay        float64
deathCountry     object
deathState       object
deathCity        object
nameFirst        object
nameLast         object
nameGiven        object
weight          float64
height          float64
bats             object
throws           object
debut            object
finalGame        object
retroID          object
bbrefID          object
playerID         object
yearID            int64
stint             int64
teamID           object
lgID             object
G                 int64
AB                int64
R                 int64
H                 int64
2B                int64
3B                int64
HR                int64
RBI             float64
SB              float64
CS              float64
BB                int64
SO              float64
IBB             

In [132]:
df_huskiesBatters = df_batters.loc[df_batters.BMI >= 34.55]

In [133]:
df_huskiesBatters.sort_values('BMI').describe()

,birthYear,birthMonth,birthDay,deathYear,deathMonth,deathDay,weight,height,yearID,stint,...,BB,SO,IBB,HBP,SH,SF,GIDP,KG,meters,BMI
count,26.000000,25.000000,25.000000,5.00000,5.000000,5.000000,26.000000,26.000000,26.000000,26.000000,...,26.000000,22.000000,0.0,24.000000,17.000000,0.0,1.0,26.000000,26.000000,26.000000
mean,1969.153846,5.800000,17.520000,1952.40000,7.600000,8.600000,269.961538,72.115385,1901.730769,1.115385,...,9.884615,14.909091,NaN,0.875000,6.000000,NaN,0.0,122.452394,1.831731,36.408324
std,40.878055,3.316625,8.529947,49.45503,3.646917,7.602631,30.180100,4.348121,14.177610,0.325813,...,16.310308,13.606403,NaN,1.650099,8.951257,NaN,NaN,13.689452,0.110442,1.728873
min,1853.000000,1.000000,1.000000,1891.00000,2.000000,2.000000,155.000000,55.000000,1872.000000,1.000000,...,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,70.306760,1.397000,34.622243
25%,1977.000000,4.000000,11.000000,1915.00000,6.000000,2.000000,261.250000,71.000000,1890.250000,1.000000,...,1.000000,3.250000,NaN,0.000000,0.000000,NaN,0.0,118.500910,1.803400,34.891189
50%,1983.500000,6.000000,18.000000,1966.00000,9.000000,6.000000,270.000000,72.000000,1903.500000,1.000000,...,3.500000,11.000000,NaN,0.000000,2.000000,NaN,0.0,122.469840,1.828800,35.875979
75%,1989.000000,8.000000,24.000000,1975.00000,10.000000,14.000000,283.750000,75.000000,1912.000000,1.000000,...,9.250000,25.250000,NaN,1.000000,7.000000,NaN,0.0,128.706730,1.905000,37.928366
max,1998.000000,12.000000,30.000000,2015.00000,11.000000,19.000000,320.000000,78.000000,1924.000000,2.000000,...,68.000000,43.000000,NaN,6.000000,33.000000,NaN,0.0,145.149440,1.981200,40.292666


In [134]:
df_huskiesPitchers = df[df_pitchers.BMI > 34.55]

NameError: name 'df' is not defined

In [ ]:
df_huskiesPitchers.head()

In [ ]:
df_huskies = pd.merge(df_huskiesBatters, df_huskiesPitchers, how='right', on='playerID')

In [ ]:
df_huskies.columns

In [ ]:
sns.jointplot(data=df_simple, x="height", y="weight", kind = "reg", truncate = False)

In [ ]:
sns.choose_diverging_palette()

In [ ]:

# Compute the correlation matrix
corr = df_huskiesBatters.corr(method="spearman")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(290, 10, n=40, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    center=0,
    square=True,
    linewidths=0.25,
    cbar_kws={"shrink": .5},
)



In [ ]:
corr_mat = df.corr().stack().reset_index(name="correlation")